In [1]:
import findspark
findspark.init('C:\spark-3.4.0-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
import pandas as pd
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
df_zone_schema = types.StructType([
    types.StructField("LocationID",types.IntegerType(),True),
    types.StructField("Borough",types.StringType(),True),
    types.StructField("Zone",types.StringType(),True),
    types.StructField("service_zone",types.StringType(),True),
])
df_zone_lookup = spark.read \
        .option("header", "true") \
        .schema(df_zone_schema) \
        .csv('file/zone_lookup.csv')

In [6]:
df_green = spark.read.options(inferSchema = 'True', header = 'True').csv('file\green_tripdata_2021-01.csv')
df_yellow = spark.read.options(inferSchema = 'True', header = 'True').csv('file\yellow_tripdata_2021-01.csv')

df_green.createOrReplaceTempView ('taxi_green')
df_yellow.createOrReplaceTempView ('taxi_yellow')

df_zone_lookup.createOrReplaceTempView('zone_lookup')

In [7]:
df_zone_lookup.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [8]:
df_zone_lookup.printSchema()
df_green.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_t

In [9]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tg.lpep_pickup_datetime) AS hour, 
    tg.PULocationID AS zoneIDPU,
    zl.Zone AS zone_name_PU,
    tg.DOLocationID AS zoneIDDO,
    zl1.Zone AS zone_name_DO,
    SUM(tg.total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    taxi_green tg
    Left JOIN zone_lookup zl
    on tg.PULocationID = zl.LocationID  
    Left JOIN zone_lookup zl1
    on tg.DOLocationID = zl1.LocationID 
    
where tg.lpep_pickup_datetime >= '2020-01-01 00:00:00'

GROUP BY
    hour, zoneIDPU,zoneIDDO,zone_name_PU,zone_name_DO
    
Order by hour asc
""")

In [10]:
df_green_revenue.show()

+-------------------+--------+--------------------+--------+--------------------+------+--------------+
|               hour|zoneIDPU|        zone_name_PU|zoneIDDO|        zone_name_DO|amount|number_records|
+-------------------+--------+--------------------+--------+--------------------+------+--------------+
|2020-12-31 23:00:00|     168|Mott Haven/Port M...|      75|   East Harlem South|   9.3|             1|
|2021-01-01 00:00:00|      76|       East New York|     108|           Gravesend| 36.01|             1|
|2021-01-01 00:00:00|     259|  Woodlawn/Wakefield|     116|    Hamilton Heights|  29.0|             1|
|2021-01-01 00:00:00|      42|Central Harlem North|      41|      Central Harlem|   7.3|             1|
|2021-01-01 00:00:00|      47|  Claremont/Bathgate|      51|          Co-Op City| 36.71|             1|
|2021-01-01 00:00:00|     165|             Midwood|      21|    Bensonhurst East| 22.87|             1|
|2021-01-01 00:00:00|     225|  Stuyvesant Heights|     225|  St

In [11]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', ty.tpep_pickup_datetime) AS hour, 
    ty.PULocationID AS zoneIDPU,
    zl.Zone AS zone_name_PU,
    ty.DOLocationID AS zoneIDDO,
    zl1.Zone AS zone_name_DO,
    SUM(ty.total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    taxi_yellow ty
    Left JOIN zone_lookup zl
    on ty.PULocationID = zl.LocationID  
    Left JOIN zone_lookup zl1
On ty.DOLocationID = zl1.LocationID 

where ty.tpep_pickup_datetime >= '2020-01-01 00:00:00'

GROUP BY
    hour, zoneIDPU,zoneIDDO,zone_name_PU,zone_name_DO
Order by hour 
""")

In [12]:
df_yellow_revenue.show()

+-------------------+--------+--------------------+--------+--------------------+------+--------------+
|               hour|zoneIDPU|        zone_name_PU|zoneIDDO|        zone_name_DO|amount|number_records|
+-------------------+--------+--------------------+--------+--------------------+------+--------------+
|2020-12-31 13:00:00|     170|         Murray Hill|     226|           Sunnyside| 20.38|             1|
|2020-12-31 18:00:00|      90|            Flatiron|     229|Sutton Place/Turt...| 16.56|             1|
|2020-12-31 18:00:00|     142| Lincoln Square East|      68|        East Chelsea|  14.3|             1|
|2020-12-31 18:00:00|      48|        Clinton East|     239|Upper West Side S...| 14.04|             1|
|2020-12-31 18:00:00|      48|        Clinton East|     238|Upper West Side N...|  12.8|             1|
|2020-12-31 18:00:00|      48|        Clinton East|     243|Washington Height...|  28.3|             1|
|2020-12-31 18:00:00|     142| Lincoln Square East|     233| UN/

In [14]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [15]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zoneIDPU'], how='outer')

In [16]:
df_join.show()

+-------------------+--------+------------+--------+--------------------+------------+--------------------+--------------------+--------+--------------------+-------------+---------------------+
|               hour|zoneIDPU|zone_name_PU|zoneIDDO|        zone_name_DO|green_amount|green_number_records|        zone_name_PU|zoneIDDO|        zone_name_DO|yellow_amount|yellow_number_records|
+-------------------+--------+------------+--------+--------------------+------------+--------------------+--------------------+--------+--------------------+-------------+---------------------+
|2020-12-31 13:00:00|     170|        null|    null|                null|        null|                null|         Murray Hill|     226|           Sunnyside|        20.38|                    1|
|2020-12-31 18:00:00|     142|        null|    null|                null|        null|                null| Lincoln Square East|     233| UN/Turtle Bay South|        17.16|                    1|
|2020-12-31 18:00:00|    